In [1]:
import cv2
import numpy as np
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QVBoxLayout, QWidget
from PyQt5.QtCore import Qt, QTimer

In [2]:
class ImageDisplay(QWidget):
    def __init__(self, image):
        super().__init__()

        self.label = QLabel()
        self.label.setAlignment(Qt.AlignCenter)
        self.label.setPixmap(self.get_pixmap(image))

        layout = QVBoxLayout()
        layout.addWidget(self.label)
        self.setLayout(layout)

    def get_pixmap(self, image):
        height, width, channel = image.shape
        bytes_per_line = 3 * width
        q_image = QImage(image.data, width, height, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(q_image)

In [3]:
def main():
    try:
        # Charger les deux images que vous souhaitez recalibrer
        image1 = cv2.imread('20230323_191223_BRS_L_UP_11.369_00.png.jpg', cv2.IMREAD_GRAYSCALE)
        image2 = cv2.imread('20230406_060421_BRS_L_UP_11.369_00.png.jpg', cv2.IMREAD_GRAYSCALE)

        if image1 is None or image2 is None:
            print("Erreur de chargement d'image. Vérifiez les chemins d'accès.")
            sys.exit(1)

    except Exception as e:
        print(f"Une erreur s'est produite : {e}")
        sys.exit(1)

In [4]:
    # Réduire le nombre de points d'intérêt
    orb = cv2.ORB_create(nfeatures=500)
    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

    # Réduire le nombre de correspondances
    num_good_matches = 50
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)[:num_good_matches]

    # Extraire les points correspondants des deux images
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches])
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches])

    if len(src_pts) < 4 or len(dst_pts) < 4:
        print("Pas assez de points pour estimer la transformation.")
        sys.exit(1)
    else:
        # Estimation de la transformation
        transformation_matrix, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        # Appliquer la transformation à l'image 1 pour la recalibrer sur l'image 2
        height, width = image2.shape
        aligned_image = cv2.warpPerspective(image1, transformation_matrix, (width, height))

        # Afficher les images originales et recalées avec PyQt
        app = QApplication(sys.argv)
        display_widget = ImageDisplay(aligned_image)
        display_widget.show()
        sys.exit(app.exec_())

if __name__ == "__main__":
    main()

IndentationError: expected an indented block (3081959590.py, line 34)